In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
from PIL import Image
from PIL import __version__
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import matplotlib.pyplot as plt
# Device configuration
device = torch.device('cuda:0')

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('board/fnm_exp1')

#!tensorboard --logdir I:\tensorboard

In [3]:
try:
    from PIL import __version__
except ImportError:
    from PIL import PILLOW_VERSION as __version__

In [4]:
import os
import numpy as np
import tensorflow as tf

In [5]:
from math import floor

In [6]:
use_bias = False
front_path = './images/fig1'
profile_path = '.images/fig2'
lambda_l1 = 0.001 #'weight of the loss for L1 texture loss') # 0.001
lambda_fea=100 #'weight of the loss for face model feature loss')
lambda_reg= 1e-5# 'weight of the loss for L2 regularitaion loss')
lambda_gan= 1# 'weight of the loss for gan loss')
lambda_gp=10# 'weight of the loss for gradient penalty on parameter of D')

# For training
dataset_size=  1000# 'dataset path')  # casia_aligned_250_250_jpg
profile_list=''# 'train profile list')
front_path=''#front data path')
front_list=''# 'train front list')
test_path=''# 'front data path')
is_train=True# 'train or test')
is_finetune= False# 'finetune') # False, True
face_mode='resnet50.npy'# 'face model path')
checkpoint='checkpoint/fnm'# 'checkpoint directory')
summary_dir= 'log/fnm'# 'logs directory')
checkpoint_ft='checkpoint/fnm/ck-09'#'finetune or test checkpoint path')
batch_size= 4# 'batch size')#was 16
BS = batch_size
epoc=10 # 'epoch')
critic= 1 #'number of D training times')
save_freq= 1000 # 'the frequency of saving model')
lr=1e-4# 'base learning rate')
beta1=0. # 'beta1 momentum term of adam')
beta2=0.9 # 'beta2 momentum term of adam')
stddev= 0.02 # 'stddev for W initializer')
use_bias=False # 'whether to use bias')
results='results/fnm' # 'path for saving results') #
############################
############################
#   environment setting    #
############################
device_id='3,4'# 'device id')
ori_height=224 # 'original height of profile images')
ori_width=224 # 'original width of profile images')
height= 224 #'height of images') # do not modified
width= 224 # 'width of images') # do not modified
CHANNEL=3 # 'channel of images')
num_threads=8 # 'number of threads of enqueueing examples')


# Resnet50 model trained on VGGFace2 dataset

In [7]:
from Resnet50_ft_dag import resnet50_ft_dag


In [8]:
weights_path= "./resnet50_ft_dag.pth"
Model = resnet50_ft_dag(weights_path=weights_path).to(device)
for p in Model.parameters():
        p.require_grad = False

In [9]:
writer = SummaryWriter("FNM", comment = "19.03.2020")

# Batch normalisation

In [10]:
# class Batch_norm(nn.Module):
#     def __init__(self, in_channels, epsilon=1e-5, momentum = 0.9, is_train = False):
#         super(Batch_norm, self).__init__()
#         # self.mean  = mosv_dict['mean']
#         self.norm = nn.BatchNorm2d(in_channels, eps = epsilon,  momentum = momentum, track_running_stats = is_train)
#         #self.scale = mosv_dict['scale']
#         #self.variance = mosv_dict['variance']
#         #self.epsilon = 1e-5
#     def forward(self, x):
#         norm = self.norm(x)
#         return norm
            
    

In [11]:
def Batch_norm(in_channels, epsilon=1e-5, momentum = 0.9, is_train = False):
    return nn.BatchNorm2d(in_channels, eps = epsilon,  momentum = momentum, track_running_stats = is_train)

In [12]:
NORM = Batch_norm

# Options: conv2d, res_block 

In [13]:
def calc_conv_pad(input_size, output_size, filter_size = 3, stride = 2):
    return max(0, floor((stride * (output_size - 1) + filter_size - input_size)/2 + 0.5))

In [14]:
def calc_deconv_pad(input_size, output_size, filter_size = 3, stride = 2):
    return max(0, floor((stride * (input_size - 1) + filter_size - output_size)/2 + 0.5))

In [15]:
def conv2d( in_channels , out_channels , kernel_size = 3 , strides = 1  , 
           padding  =  None,  pad_input = None,
           bias = use_bias,dilation_rate = 1, activation = None):
    layers = []
    if padding == None:
        if pad_input is not None:
            padding = calc_conv_pad(pad_input , 2*pad_input, kernel_size , strides)
        else:
            padding = 0
    conv = nn.Conv2d( in_channels , out_channels , bias=bias, kernel_size = kernel_size, stride=strides, padding=padding, dilation=dilation_rate )
    layers.append( conv )
    if activation is not None:
        layers.append( activation )
    return nn.Sequential( *layers )

def deconv2d(in_channels , out_channels , kernel_size = 3 , strides = 1  , 
             padding  = None, pad_input = None,
             bias= use_bias, dilation_rate = 1,
            activation = None):
    if padding == None:
        if pad_input is not None:
            padding = calc_deconv_pad(pad_input , 2*pad_input, kernel_size , strides)
        else:
            padding = 0
    layers = [nn.ConvTranspose2d(in_channels, out_channels, kernel_size =  kernel_size, stride=strides, 
                      padding=padding, bias=bias)]
    if activation is not None:
        layers.append( activation )
    return nn.Sequential( *layers )
    

In [16]:
class res_block(nn.Module):
    def __init__(self,  in_channels , 
                 out_channels , kernel_size = 3, 
                 stride = 1  , padding  =  None, pad_input = None,
                 bias = use_bias,  norm = NORM, activation2 = nn.ReLU, activation = None,):
        super(res_block, self).__init__()
        if padding == None:
            if pad_input is not None:
                padding = calc_conv_pad(pad_input , pad_input, kernel_size , stride)
            else: 
                padding = 0
        self.out_channels = out_channels
        if activation is not None:
            self.activation = activation(out_channels)
        else:
            self.activation = activation
        self.activation2 = activation2(out_channels)
        convs = [conv2d(in_channels , out_channels , kernel_size , stride , padding, bias, activation = self.activation), 
             nn.BatchNorm2d( in_channels ), 
             nn.ReLU(out_channels), 
             conv2d(out_channels, out_channels, kernel_size, stride, padding, bias, activation = self.activation),
             norm(in_channels)]
        self.layers = nn.Sequential(*convs)
        #независимо от передаваемого параметра NORM норма всегда Batch_norm
    def forward(self, x):
        return self.activation2(self.layers(x) + x)

# Load data

In [17]:
from random import random

In [18]:
import glob
import shutil
import os

src_dir = "your/source/dir"
dst_dir = "your/destination/dir"
profile_set = set()
profile_list = []
front_set = set()
front_list = []
#frontFile = open("./TPdata/train_data/front_list.txt", "w")
#profileFile = open("./TPdata/train_data/profile_list.txt", "w")

def relocate_data(src_dir, src_dir_front, src_dir_profile, variety = 3):
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.png")):
        tmp = jpgfile.split('\\')[-1].split('.')[0].split('_')
        ttmp = tmp[0]+ "_"+tmp[2]
        if (len(tmp) > 6):
            if ttmp not in profile_set or random()< 1/2:
                shutil.copy(jpgfile, src_dir_profile)
                profileFile.write(jpgfile+"\n")
                #profile_list.append(jpgfile)
                profile_set.add(ttmp)
        else:
            if ttmp not in front_set or random()< 3/7:
                shutil.copy(jpgfile, src_dir_front)
                frontFile.write(jpgfile+"\n")
                #front_list.append(jpgfile)
                front_set.add(ttmp)
                

In [19]:
src_dir = "C:/Users/d.baranovska/Downloads/combined_data"
src_dir_front = "./TPdata/train_data/front"
src_dir_profile = "./TPdata/train_data/profile"
#relocate_data(src_dir, src_dir_front, src_dir_profile)

In [20]:
#frontFile.close()
#profileFile.close()

In [21]:
frontFile = open("./TPdata/train_data/front_list.txt", "r+")
profileFile = open("./TPdata/train_data/profile_list.txt", "r+")
front_list = list(map(lambda x: x.strip(), frontFile.readlines()))
profile_list = list(map(lambda x: x.strip(),profileFile.readlines()))
frontFile.close()
profileFile.close()

In [22]:
print(len(front_list))
print(len(profile_list))

1353
1533
1353
1533


In [23]:
from torch.utils.data import Dataset
from PIL import Image

In [24]:
PIC_SIZE = (224, 224)
class TrainDataset(Dataset):
    def __init__(self, img_list):
        super(type(self), self).__init__()
        self.img_list = img_list
    def __len__(self):
        return len(self.img_list)
    def __getitem__(self, idx):
        #229_01_01_200_08_cropped_test.png
        img_name = self.img_list[idx]
        with Image.open(img_name) as i:
            im = i.resize(PIC_SIZE)
            our_tensor = transforms.ToTensor()(im)
        return our_tensor

# Generator 

In [25]:
f7_shape = [7, 7, 2048]

#### second variant

In [26]:
class Generator(nn.Module):
    def __init__(self): #, profile, front, train):
        super(Generator, self).__init__() 
#         self.face_model = Resnet50_ft_dag()
#         self.feature_p = self.face_model.forward(profile)
#         self.feature_f = self.face_model.forward(front)
#         self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
#         self.is_train = train
        
        self.conv1 = nn.Sequential(conv2d(2048, 512, kernel_size=1, strides = 1), NORM(512, is_train), nn.ReLU())
    
        self.res1_1 = res_block(512, 512, norm = NORM, pad_input = 7)
        self.res1_2 = res_block(512, 512, norm = NORM, pad_input = 7)
        self.res1_3 = res_block(512, 512, norm = NORM, pad_input = 7)
        self.res1_4 = res_block(512, 512, norm = NORM, pad_input = 7)
        self.dconv2 = nn.Sequential(deconv2d(512, 256, kernel_size=4, strides = 2, pad_input = 7), NORM(256), nn.ReLU())
        self.res2 = res_block(256, 256, norm = NORM, pad_input = 14)
        self.dconv3 = nn.Sequential(deconv2d(256, 128, kernel_size=4, strides = 2, pad_input = 14) , NORM(128), nn.ReLU())
        self.res3 = res_block(128, 128, norm = NORM, pad_input = 28)
        self.dconv4 = nn.Sequential(deconv2d(128, 64, kernel_size=4, strides = 2, pad_input = 28), NORM(64), nn.ReLU())
        self.res4 = res_block(64, 64, norm = NORM, pad_input = 56)
        self.dconv5 = nn.Sequential(deconv2d(64, 32, kernel_size=4, strides = 2, pad_input = 56), NORM(32), nn.ReLU())
        self.res5 = res_block(32, 32, norm = NORM, pad_input = 112)
        self.dconv6 = nn.Sequential(deconv2d(32, 32, kernel_size=4, strides = 2, pad_input =112),  NORM(32), nn.ReLU())
        self.res6 = res_block(32, 32, norm = NORM,  pad_input = 224)
        self.gen = nn.Sequential(conv2d(32, 3,  kernel_size=1, strides = 1), nn.Tanh())
        
    def forward(self, feature):
        featt7 = feature[0]
        feat7 = self.conv1(featt7)
        pool5 = feature[1]
        res1_1 = self.res1_1(feat7)
        res1_2 = self.res1_2(res1_1)
        res1_3 = self.res1_3(res1_2)
        res1_4 = self.res1_2(res1_3)
        dconv2 = self.dconv2(res1_4)
        res2 = self.res2(dconv2)
        dconv3 = self.dconv3(res2)
        res3 = self.res3(dconv3)
        dconv4 = self.dconv4(res3)
        res4 = self.res4(dconv4)
        dconv5 = self.dconv5(res4)
        res5 = self.res5(dconv5)
        dconv6 = self.dconv6(res5)
        res6 = self.res6(dconv6)
        gen = self.gen(res6)
        return (gen + 1)* 127.5

In [27]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        pad = 1
        self.h0_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h0_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h0_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h0_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h0_4 = nn.Sequential(conv2d(256, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h0_5 = nn.Linear(12544, 1)
        
        self.h1_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h1_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h1_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h1_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h1_4 = nn.Linear(6144, 1)
        
        self.h2_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h2_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h2_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h2_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        #self.h2_4 = nn.Linear(self.h2_3.out_features, 1)
        self.h2_4 = nn.Linear(3840, 1)
        
        self.h3_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h3_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h3_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h3_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h3_4 = nn.Linear(2560, 1)
        
        self.h4_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h4_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h4_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h4_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h4_4 = nn.Linear(16384, 1)
    
    
    def forward(self, images):
        eyes = images[0:BS,0:CHANNEL,64:100,50:174] #[BS,36,124,CHANNEL])
        nose =  images[0:BS,0:CHANNEL,75:140,90:134]#tf.slice(images, [0,75,90,0], [BS,65,44,CHANNEL])
        mouth = images[0:BS,0:CHANNEL,140:170,75:149]#tf.slice(images, [0,140,75,0], [BS,30,74,CHANNEL])
        face = images[0:BS,0:CHANNEL,64:180,50:174]#[BS,116,124,CHANNEL])
        h0_0 = self.h0_0(images)
        h0_1 = self.h0_1(h0_0)
        h0_2 = self.h0_2(h0_1)
        h0_3 = self.h0_3(h0_2)
        h0_4 = self.h0_4(h0_3)
        h0_5 = self.h0_5(torch.reshape(h0_4, [BS, -1]))
        
        h1_0 = self.h1_0(eyes)
        h1_1 = self.h1_1(h1_0)
        h1_2 = self.h1_2(h1_1)
        h1_3 = self.h1_3(h1_2)
        h1_4 = self.h1_4(torch.reshape(h1_3, [BS, -1]))
        
        h2_0 = self.h2_0(nose)
        h2_1 = self.h2_1(h2_0)
        h2_2 = self.h2_2(h2_1)
        h2_3 = self.h2_3(h2_2)
        h2_4 = self.h2_4(torch.reshape(h2_3, [BS, -1]))
        
        h3_0 = self.h3_0(mouth)
        h3_1 = self.h3_1(h3_0)
        h3_2 = self.h3_2(h3_1)
        h3_3 = self.h3_3(h3_2)
        h3_4 = self.h3_4(torch.reshape(h3_3, [BS, -1]))
        
        h4_0 = self.h4_0(face)
        h4_1 = self.h4_1(h4_0)
        h4_2 = self.h4_2(h4_1)
        h4_3 = self.h4_3(h4_2)
        h4_4 = self.h4_4(torch.reshape(h4_3, [BS, -1]))
        return torch.cat((h0_5, h1_4, h2_4, h3_4, h4_4))

# G and D

In [28]:
#built network
z_dim = 224*224
mnist_dim = 224*224
#mnist_dim = train_dataset_front.train_data.size(1) * train_dataset.train_data.size(2)

G = Generator().to(device)
D = Discriminator().to(device)

In [29]:
print(torch.__version__)

1.2.0+cu92
1.2.0+cu92


In [30]:
criterion = nn.MSELoss(reduction='mean')

# optimizer
lr = 0.0005 
G_optimizer = optim.Adam(G.parameters(), lr)#,betas=(beta1, beta2), weight_decay=0)
D_optimizer = optim.Adam(D.parameters(), lr)#, betas=(beta1, beta2), weight_decay=0)


# Loss functions

In [31]:
Losses = []
D_losses, G_losses = [], []
D_finalLosses, G_finalLosses = [], []

In [32]:
def reverse(x, dim):
    dim = x.dim() + dim if dim < 0 else dim
    return x[tuple(slice(None, None) if i != dim
             else torch.arange(x.size(i)-1, -1, -1).long()
             for i in range(x.dim()))]

In [33]:
epsilon = 1e-9

In [34]:
global a 
a = 0
global epoch_num
epoch_num = 0

In [35]:

#print("Gen.gen parametrs", torch.sum(i(*G.parameters())).item())

In [36]:
def loss(profile, front):
    #=======================Train the generator=======================#
    G_optimizer.zero_grad()
    D_optimizer.zero_grad()
    Model.eval()
    for p in Model.parameters():
        p.require_grad = False
    feature_p = Model.forward(profile) # G_enc(x)
    feature_f = Model.forward(front) # G_enc(y)
#     feature_p = [i for i in feature_p]
#     feature_f = [i for i in feature_f]
#     for i in feature_p:
#         i = i.to(device)
#     for i in feature_f:
#         i = i.to(device)
    gen_p = G.forward(feature_p) # ~x
    gen_f = G.forward(feature_f) # ~y
    if a % 30 == 0:
        print('')
        print(a)
        print(torch.sum(feature_p[0] - feature_f[0]))
        print(torch.sum(gen_p - gen_f))
        print("gen_p 0 - 1", torch.sum(gen_p[0] -  gen_p[1]))
        print("front 0 - 1", torch.sum(front[0] -  front[1]))
        #print("Model conv5_3_3x3 sum =", torch.sum(*Model.conv5_3_3x3.parameters()).item())
        #print("Gen.dconv6 parameters sum", torch.sum(*G.dconv6.parameters()).item())
        print("Gen.gen params sum", torch.sum(*G.gen.parameters()).item())
        #print("Dis.h0_5 params sum", torch.sum(*D.h0_5.parameters()).item())
        #save_image(feature_p[0].int(), "./TPdata/train_data/processed/" + str(a)+ "_feature_p" + ".png")
        save_image(profile, "I:/train_data/processed/" + str(epoch_num)+"_"+ str(a)+"_profile" + ".png")
        save_image(gen_f.int(), "I:/train_data/processed/" + str(epoch_num)+"_"+ str(a)+ "_gen_f_all" + ".png")
        save_image(gen_p.int(), "I:/train_data/processed/" +str(epoch_num)+"_"+ str(a)+  "_gen_p_" + ".png")
    #for i in gen_p: writer.add_image('~x - gen_p', i)
    #for i in gen_f: writer.add_image('~y - gen_f', i)
#     dr = torch.cat(D(front))
#     df1 = torch.cat(D(gen_p))
#     df2 = torch.cat(D(gen_f))
    dr = D(front)
    df1 = D(gen_p)
    df2 = D(gen_f)
    feature_gen_p = Model.forward(gen_p) #G_enc(~x)
    feature_gen_f = Model.forward(gen_f) #G_enc(~y)
    
    pool5_p_norm = feature_p[-1]/(torch.norm(feature_p[-1],dim = 1, keepdim = True) + epsilon)
    pool5_f_norm = feature_f[-1]/(torch.norm(feature_f[-1],dim = 1, keepdim = True) + epsilon)
    
    pool5_gen_p_norm = feature_gen_p[-1]/(torch.norm(feature_gen_p[-1],dim = 1, keepdim = True) + epsilon)
    pool5_gen_f_norm = feature_gen_f[-1]/(torch.norm(feature_gen_f[-1],dim = 1, keepdim = True) + epsilon)
    
     # 1. Frontalization Loss: L1-Norm
    front_loss = torch.mean(torch.sum(torch.abs(front/255. - gen_f/255.), [1,2,3]))
    
    # 2. identity perseption loss l2-norm
    feature_distance = 0.5*(1 - torch.sum(torch.mul(pool5_p_norm, pool5_gen_p_norm), [1])) + \
                                0.5*(1 - torch.sum(torch.mul(pool5_f_norm, pool5_gen_f_norm), [1]))
    feature_loss = torch.mean(feature_distance)
    #Losses.append(feature_loss.item())
    #.
    #trainable var
    #all_vars = torch.autograd.Variables()
    #vars_gen = Variable(G.parameters, requires_grad= True)
    #vars_dis = Variable(D.parameters, requires_grad= True)
    #vars_gen = weights_list=[var for var in self.vars_gen if 'kernel' in var.name]
    
    
    # 3. Regulation loss
    reg_gen = criterion(gen_p, profile)
    reg_dis = criterion(gen_f, front)
#     loss = nn.MSELoss(lambda_reg, reduction='mean')
#     #reg_gen = loss(weights_list=[var for var in vars_gen]) # if 'kernel' in var.name
#     #reg_dis = loss(weights_list=[var for var in vars_dis])# if 'kernel' in var.name
#     target = torch.zeros(G.parameters.shape)
#     reg_gen = loss(G.parameters, target)
#     reg_dis = loss(D.parameters, target)
#     G_losses.append(reg_gen)
#     D_losses.append(reg_dis)
    
    
    # 4. Adversarial Loss
    d_loss = torch.mean(torch.sum(df1)*0.5 + torch.sum(df2)*0.5 - torch.sum(dr)) / 5
    g_loss = -torch.mean(torch.sum(df1)*0.5 + torch.sum(df2)*0.5) / 5
    #D_losses.append(d_loss.item())
    #G_losses.append(g_loss.item())
    
    # 5. Symmetric Loss - not applied
    #mirror_p = reverse(gen_p, dim=[2])
    #sym_loss = torch.mean(torch.sum(torchf.abs(mirror_p/225. - gen_p/255.), [1,2,3]))
    
    # Gradient Penalty #
    alpha = torch.rand([front.size(0), 1, 1, 1]).to(device)
    inter = (alpha*front + ((1 - alpha) * gen_p )).requires_grad_(True).to(device)
    d = D(inter)
    fake = Variable(torch.Tensor(BS*5, 1).fill_(1.0), requires_grad=False).to(device)
    grad = torch.autograd.grad(d, inter, grad_outputs=fake,
    retain_graph=True,
    create_graph=False,
    only_inputs=True,)[0]
    #grad = d.grad
    grad = grad.view(grad.size(0), -1)
    gradient_penalty = ((grad.norm(2, dim=1)-1)**2).mean()
      
     # 6. Drift Loss - not applied
    #drift_loss = 0
    #torch.mean(torch.add(torch.square(df)) + torch.add(torch.square(dr))) / 10
    
    Gen_loss = lambda_l1*front_loss + lambda_fea*feature_loss +(lambda_gan)*g_loss + reg_gen
    Dis_loss = lambda_gan* d_loss + reg_dis+lambda_gp*gradient_penalty# + torch.tensor(lambda_gp) * gradient_penalty
    
    #Gen_loss = Gen_loss.to(device)
    #Dis_loss = Gen_loss.to(device)
    #G_finalLosses.append(Gen_loss.item())
    #D_finalLosses.append(Dis_loss.item())
#     Gen_loss.backward()
#     Dis_loss.backward()
    
    (Gen_loss + Dis_loss).backward()
    

    
    G_optimizer.step()
    D_optimizer.step()
#     G_optimizer.zero_grad()
#     D_optimizer.zero_grad()
    if a < 30:
        writer.add_scalar('Gen_loss', Gen_loss , epoch_num*30 + a)
        writer.add_scalar('Dis_loss', Dis_loss , epoch_num*30 + a)
        writer.add_scalar('front_loss',  front_loss , epoch_num*30 + a)
        writer.add_scalar('feature_loss',  feature_loss , epoch_num*30 + a)
        writer.add_scalar('g_loss',  g_loss , epoch_num*30 + a)
        writer.add_scalar('d_loss',  d_loss , epoch_num*30 + a)
        writer.add_scalar(' gradient_penalty',gradient_penalty, epoch_num*30 + a )
        writer.add_image('gen_p', gen_p[0], epoch_num*100 + a )
        writer.add_image('gen_f', gen_f[0], epoch_num*100 + a)
        writer.add_image('front',  front[0], epoch_num*100 + a)
        writer.add_image('profile',  profile[0], epoch_num*100 + a)
        writer.add_image('gen_p', gen_p[1], epoch_num*100 + a)
        writer.add_image('gen_f',  gen_f[1], epoch_num*100 + a)
        writer.add_image('front', front[1], epoch_num*100 + a)
        writer.add_image('profile',  profile[1], epoch_num*100 + a)
        writer.add_image('gen_p', gen_p[2], epoch_num*100 + a)
        writer.add_image('gen_f', gen_f[2], epoch_num*100 + a)
        writer.add_image('front', front[2], epoch_num*100 + a)
        writer.add_image('profile',  profile[2], epoch_num*100 + a)
    return Gen_loss,  Dis_loss, front_loss, feature_loss, g_loss, d_loss
    
#     z = Variable(torch.randn(bs, z_dim).to(device))
#     y = Variable(torch.ones(bs, 1).to(device))

#     G_output = G(z)
#     D_output = D(G_output)
#     G_loss = criterion(D_output, y)

#     # gradient backprop & optimize ONLY G's parameters
#     G_loss.backward()
#     G_optimizer.step()
        
#     return G_loss.data.item()

In [37]:
sum(p.numel() for p in G.parameters() if p.requires_grad)


24322144

24322144

In [38]:
# Model conv5_3_3x3 sum = -1227.2626953125

# Main

In [39]:
x = torch.rand([BS, 3, 224, 224])
y = torch.rand([BS, 3, 224, 224])
t = Model(x.to(device))
x_gen = G(t)
#reg_gen = criterion(x_gen, x)
#print(reg_gen)
print(x_gen.size())
print("Model conv5_3_3x3 sum =", torch.sum(*Model.conv5_3_3x3.parameters()).item())

torch.Size([4, 3, 224, 224])
Model conv5_3_3x3 sum = -1227.2626953125
torch.Size([4, 3, 224, 224])
Model conv5_3_3x3 sum = -1227.2626953125


In [40]:
x_dis = D(x_gen)
print(x_dis[0].size(), x_dis[1].size(), x_dis[2].size(), x_dis[3].size(), x_dis[4].size())

torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1])
torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1]) torch.Size([1])


In [41]:
Train_front_data = TrainDataset(front_list)
Train_profile_data = TrainDataset(profile_list)

In [42]:
train_loader_front = torch.utils.data.DataLoader(dataset=Train_front_data, batch_size=BS, shuffle=True, drop_last=True)
train_loader_profile = torch.utils.data.DataLoader(dataset=Train_profile_data, batch_size=BS, shuffle=True, drop_last=True)
#test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [43]:
d = list(enumerate(train_loader_front))
print(d[3][1].size())

torch.Size([4, 3, 224, 224])
torch.Size([4, 3, 224, 224])


In [44]:
(profile_list[0].split('\\')[-1].split('.')[0])

'201_01_01_010_08_cropped_test'

'201_01_01_010_08_cropped_test'

In [ ]:
n_epoch = 200
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    d = list(enumerate(train_loader_front))
    for j in range(len(d)):
        d[j] = (j, torch.autograd.Variable(d[j][1].to(device), requires_grad = False))
    for batch_idx, x in enumerate(train_loader_profile):
        for k in range(len(x)):
            x[k] = torch.autograd.Variable(x[k].to(device), requires_grad = False)
        x = x.to(device)
        if batch_idx >= len(d):
            break
        Gen_loss,  Dis_loss, front_loss, feature_loss, g_loss, d_loss = loss(x, d[batch_idx][1])
        D_losses.append(Gen_loss)
        G_losses.append(Dis_loss)
        
        a += 1
    a = 0
    epoch_num += 1 
        #print(list(map(lambda x: x.item(), [Gen_loss,  Dis_loss, front_loss, feature_loss, g_loss, d_loss])))

    
    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))
    #writer.add_histogram('Gen_losses', np.array(G_losses))
    #writer.add_histogram('Dis_losses', np.array(D_losses))


0
tensor(776.7050, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4201.6641, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-3461.1108, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(1312.5330, device='cuda:0')
Gen.gen params sum 0.048533037304878235

0
tensor(776.7050, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4201.6641, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-3461.1108, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(1312.5330, device='cuda:0')
Gen.gen params sum 0.048533037304878235

30
tensor(88.3518, device='cuda:0', grad_fn=<SumBackward0>)
tensor(961.3184, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(502.3515, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(13167.2939, device='cuda:0')
Gen.gen params sum -1.0607476234436035

30
tensor(88.3518, device='cuda:0', grad_fn=<SumBackward0>)
tensor(961.3184, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(502.3515, device='cuda:0', g


90
tensor(437.3551, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4.4780, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-11.6184, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(13670.2041, device='cuda:0')
Gen.gen params sum -3.5843148231506348

90
tensor(437.3551, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4.4780, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-11.6184, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(13670.2041, device='cuda:0')
Gen.gen params sum -3.5843148231506348

120
tensor(756.6035, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-1.1824, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(6.5282, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(1115.3723, device='cuda:0')
Gen.gen params sum -3.647961378097534

120
tensor(756.6035, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-1.1824, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(6.5282, device='cuda:0', grad_fn=<SumB


180
tensor(908.7452, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-2.4602, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(4.4243, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-32974.2422, device='cuda:0')
Gen.gen params sum -4.218525409698486

180
tensor(908.7452, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-2.4602, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(4.4243, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-32974.2422, device='cuda:0')
Gen.gen params sum -4.218525409698486

210
tensor(355.1144, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-3.9576, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-7.5235, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(9872.6318, device='cuda:0')
Gen.gen params sum -4.249306678771973

210
tensor(355.1144, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-3.9576, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-7.5235, device='cuda:0', grad_fn=<Su


270
tensor(902.2239, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-0.5447, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-0.2545, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(3361.5095, device='cuda:0')
Gen.gen params sum -4.6125593185424805

270
tensor(902.2239, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-0.5447, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-0.2545, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(3361.5095, device='cuda:0')
Gen.gen params sum -4.6125593185424805

300
tensor(1010.7150, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1.3993, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(8.2317, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-2589.5061, device='cuda:0')
Gen.gen params sum -4.635649681091309

300
tensor(1010.7150, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1.3993, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(8.2317, device='cuda:0', grad_fn=<Sum

[5/200]: loss_d: -40127.691, loss_g: 147176320.000
[5/200]: loss_d: -40127.691, loss_g: 147176320.000

0
tensor(130.4806, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-0.5673, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-1.3429, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-1425.4825, device='cuda:0')
Gen.gen params sum -4.869325637817383

0
tensor(130.4806, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-0.5673, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-1.3429, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-1425.4825, device='cuda:0')
Gen.gen params sum -4.869325637817383

30
tensor(746.3111, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-0.4015, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(2.2107, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(3016.5107, device='cuda:0')
Gen.gen params sum -4.884938716888428

30
tensor(746.3111, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-0.4015, de

...


270
tensor(847.1566, device='cuda:0', grad_fn=<SumBackward0>)
tensor(53844.2930, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(14647.1348, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(18276.9727, device='cuda:0')
Gen.gen params sum -7.591300010681152

300
tensor(542.5335, device='cuda:0', grad_fn=<SumBackward0>)
tensor(25649.8789, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-9463.0898, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-4357.0430, device='cuda:0')
Gen.gen params sum -7.5801801681518555

300
tensor(542.5335, device='cuda:0', grad_fn=<SumBackward0>)
tensor(25649.8789, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-9463.0898, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-4357.0430, device='cuda:0')
Gen.gen params sum -7.5801801681518555

330
tensor(574.0557, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-3402.0369, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-1845.0417, d

[105/200]: loss_d: -12587562.000, loss_g: 64798217732096.000
[105/200]: loss_d: -12587562.000, loss_g: 64798217732096.000

0
tensor(692.9815, device='cuda:0', grad_fn=<SumBackward0>)
tensor(11498.1641, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-3275.2065, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(14121.0713, device='cuda:0')
Gen.gen params sum -7.5832695960998535

0
tensor(692.9815, device='cuda:0', grad_fn=<SumBackward0>)
tensor(11498.1641, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-3275.2065, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(14121.0713, device='cuda:0')
Gen.gen params sum -7.5832695960998535

30
tensor(776.9681, device='cuda:0', grad_fn=<SumBackward0>)
tensor(41911.8047, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(8692.3223, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(7707.5732, device='cuda:0')
Gen.gen params sum -7.569526672363281

30
tensor(776.9681, device='cuda:0', gr


60
tensor(1059.0371, device='cuda:0', grad_fn=<SumBackward0>)
tensor(51168.2695, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-3469.9656, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-12618.0166, device='cuda:0')
Gen.gen params sum -7.555238723754883

90
tensor(1132.9609, device='cuda:0', grad_fn=<SumBackward0>)
tensor(54589.9844, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-2615.1958, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(1318.3451, device='cuda:0')
Gen.gen params sum -7.562640190124512

90
tensor(1132.9609, device='cuda:0', grad_fn=<SumBackward0>)
tensor(54589.9844, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-2615.1958, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(1318.3451, device='cuda:0')
Gen.gen params sum -7.562640190124512

120
tensor(247.8349, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-9050.9541, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-4586.4341, devi


150
tensor(342.1571, device='cuda:0', grad_fn=<SumBackward0>)
tensor(43419.3672, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-1802.5953, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(26589.3164, device='cuda:0')
Gen.gen params sum -7.603586196899414

150
tensor(342.1571, device='cuda:0', grad_fn=<SumBackward0>)
tensor(43419.3672, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-1802.5953, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(26589.3164, device='cuda:0')
Gen.gen params sum -7.603586196899414

180
tensor(700.3066, device='cuda:0', grad_fn=<SumBackward0>)
tensor(53324.0625, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-4414.1572, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(13445.7891, device='cuda:0')
Gen.gen params sum -7.6158857345581055

180
tensor(700.3066, device='cuda:0', grad_fn=<SumBackward0>)
tensor(53324.0625, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-4414.1572, de


210
tensor(984.8351, device='cuda:0', grad_fn=<SumBackward0>)
tensor(84411.8281, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(12735.7129, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-18596.7539, device='cuda:0')
Gen.gen params sum -7.605581283569336

240
tensor(1027.0403, device='cuda:0', grad_fn=<SumBackward0>)
tensor(61671.9141, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-6927.5703, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-1122.6234, device='cuda:0')
Gen.gen params sum -7.615220069885254

240
tensor(1027.0403, device='cuda:0', grad_fn=<SumBackward0>)
tensor(61671.9141, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(-6927.5703, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-1122.6234, device='cuda:0')
Gen.gen params sum -7.615220069885254

270
tensor(-82.8680, device='cuda:0', grad_fn=<SumBackward0>)
tensor(10991.9199, device='cuda:0', grad_fn=<SumBackward0>)
gen_p 0 - 1 tensor(14485.7656, 